<a href="https://colab.research.google.com/github/Huangjian2013/ai-demo/blob/main/rag/18-GraphRAG-Neo4j.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install llama-index llama-index-graph-stores-neo4j --quiet


In [ ]:
from llama_index.graph_stores.neo4j import Neo4jPropertyGraphStore
from google.colab import userdata
from llama_index.core import SimpleDirectoryReader, Document
from llama_index.core import PropertyGraphIndex
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.core.indices.property_graph import SimpleLLMPathExtractor

In [ ]:
graph_store = Neo4jPropertyGraphStore(
    username=userdata.get("NEO_USER_NAME"),
    password=userdata.get("NEO_PASSWORD"),
    url=userdata.get("NEO_URL")
)


In [ ]:
file_path = './sample_data/story.txt'
with open(file_path, "r", encoding="utf-8") as f:
    content = f.read()

# 将文件内容转换为 LlamaIndex 支持的 Document 对象
documents = [Document(text=content)]

In [ ]:
index = PropertyGraphIndex.from_documents(
    documents,
    embed_model=OpenAIEmbedding(model_name="text-embedding-ada-002"),
    kg_extractors=[
        SimpleLLMPathExtractor(
            llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0, api_key=userdata.get('OPENAI_API_KEY'))
        )
    ],
    property_graph_store=graph_store,
    show_progress=True,
)

In [ ]:
retriever = index.as_retriever(
    include_text=False,  # include source text in returned nodes, default True
)

nodes = retriever.retrieve("小镇里面都住着谁")

for node in nodes:
    print(node.text)

In [ ]:
query_engine = index.as_query_engine(include_text=True)

response = query_engine.query("小镇里面都住着谁")

print(str(response))